In [1]:
import pandas as pd
from tqdm import tqdm
import os, shutil, glob
import subprocess
from subprocess import Popen, PIPE
import sys
import re
from bs4 import BeautifulSoup
import javalang

In [8]:
def findAbsolutePathMyMachine(path, classes):
    for jClass in classes:
        #print(jClass)
        if path in jClass:
            return jClass
        
base_path = 'Copilot-Robustness-New/Ready/'
generated_instances_path = '/Users/luca/Public/Dataset/NonFullContext/'
df = pd.read_csv('Dataset/all-instances.csv',index_col="index")

In [9]:
pattern = re.compile(r"(?:(?:public|private|protected|static|final|native|synchronized|abstract|transient)+\s+)+[$_\w<>\[\]\s]*\s+[\$_\w]+\([^\)]*\)?\s*\{?[^\}]*\}?")

def extractGeneratedMethod(supposedToBeMethod, startingLine, endingLine):
    

    spanToLine = startingLine
    flagQuote = False
    flagValid = False
    counter = 0
    
    fromTo = '\n'.join(supposedToBeMethod.splitlines()[startingLine+1:])
    
    try:
        nextSignature = pattern.findall(fromTo)[0].split('\n')[0]
    except Exception:
        # if fails, then we reached the EOF
        nextSignature = ""
    
    span = endingLine - startingLine
    
    for line in supposedToBeMethod.splitlines()[startingLine:]:
                
        for character in line:
            
            if character == '"' and not flagQuote:
                flagQuote = True
            
            else:
                if character == '"' and flagQuote:
                    flagQuote = False
                
            if character == '{' and not flagQuote:
                counter += 1
            
            if character == '}' and not flagQuote:
                #print('hit')
                counter -= 1

        
       
        if counter == 0 and ''.join(line.strip().split()) != ''.join(nextSignature.strip().split()):
            flagValid = True
            break

        if ''.join(line.strip().split()) == ''.join(nextSignature.strip().split()):
            flagValid = False
            break
            
        else:
            spanToLine += 1
    
    
    flagEmpty = True
    for line in supposedToBeMethod.splitlines()[startingLine+1:spanToLine]:
        if line.strip() != '':
            flagEmpty = False
            break
    
    if flagEmpty:
        return -2
    
    if flagValid:
        return spanToLine+1
    
    if not flagValid:
        return -1
    
    return -1
    
    #return -1
    
def sanitySaveCheck(javaClass, targetString):
    if targetString in javaClass:
        return False
    else:
        return True

In [10]:
df['generatedResultOriginal'] = ["" for item in range(0,893)]
df['generatedResultEvaluator'] = ["" for item in range(0,893)]
df['generatedResultPegasus'] = ["" for item in range(0,893)]
df['generatedResultPivoting'] = ["" for item in range(0,893)]

In [11]:
resultOriginalList = []
resultEvaluatorList = []
resultsPegasusList = []
resultsPivotingList = []

for (idx,row) in tqdm(df.iterrows()):
    #print(idx)
    
    indexFolder = idx #row['index']
    startingMethodLine = int(row['spanMethod'].split('-')[0])+1
    endingMethodLine = int(row['spanMethod'].split('-')[1])
    
    
    ######################################################################################
       
    #try:
    path_to_original = generated_instances_path + str(indexFolder) + '/' + 'Original.java'
    with open(path_to_original) as f:
        originalJava = f.read()

    path_to_original_result = generated_instances_path + str(indexFolder) + '/' + 'ResultOriginal.java'
    with open(path_to_original_result) as f:
        originalJavaResult = f.read()

    if(not sanitySaveCheck(originalJavaResult, 'ResultOriginal')):
        print(indexFolder)


    endingLine = extractGeneratedMethod(originalJavaResult, startingMethodLine, endingMethodLine)
    if endingLine == -1:
        resultOriginalList.append("Not Valid")
        df.loc[indexFolder, 'generatedResultOriginal'] = "Not Valid"

    elif endingLine == -2:
        df.loc[indexFolder, 'generatedResultOriginal'] = "Empty Method"

    else:
        result = '\n'.join(originalJavaResult.splitlines()[startingMethodLine:endingLine])
        resultOriginalList.append(result)
        df.loc[indexFolder, 'generatedResultOriginal'] = result



    ######################################################################################

    path_to_evaluator = generated_instances_path + str(indexFolder) + '/' + 'PerturbedEvaluator.java'
    with open(path_to_evaluator) as f:
        perturbated_evaluator_Java = f.read()

    path_to_evaluator_result = generated_instances_path + str(indexFolder) + '/' + 'ResultPerturbedEvaluator.java'
    with open(path_to_evaluator_result) as f:
        perturbated_evaluator_Java_Result = f.read()


    if(not sanitySaveCheck(perturbated_evaluator_Java_Result, 'ResultPerturbedEvaluator')):
        print(indexFolder)


    endingLine = extractGeneratedMethod(perturbated_evaluator_Java_Result, startingMethodLine, endingMethodLine)
    #print('--> ',endingLine)
    if endingLine == -1:
        resultEvaluatorList.append("Not Valid")
        df.loc[indexFolder, 'generatedResultEvaluator'] = "Not Valid"

    elif endingLine == -2:
        df.loc[indexFolder, 'generatedResultEvaluator'] = "Empty Method"

    else:
        result = '\n'.join(perturbated_evaluator_Java_Result.splitlines()[startingMethodLine:endingLine])
        resultEvaluatorList.append(result)
        df.loc[indexFolder, 'generatedResultEvaluator'] = result



    ######################################################################################

    path_to_pegasus = generated_instances_path + str(indexFolder) + '/' + 'PerturbedPegasus.java'
    with open(path_to_pegasus) as f:
        perturbed_pegasus_Java = f.read()

    path_to_pegasus_result = generated_instances_path + str(indexFolder) + '/' + 'ResultPerturbedPegasus.java'
    with open(path_to_pegasus_result) as f:
        perturbed_pegasus_Java_Result = f.read()

    if(not sanitySaveCheck(perturbed_pegasus_Java_Result, 'ResultPerturbedPegasus')):
        print(indexFolder)

    endingLine = extractGeneratedMethod(perturbed_pegasus_Java_Result, startingMethodLine, endingMethodLine)
    #print('--> ',endingLine)
    if endingLine == -1:
        #print(' {} ----- {} '.format(indexFolder, startingMethodLine))
        resultsPegasusList.append("Not Valid")
        df.loc[indexFolder, 'generatedResultPegasus'] = "Not Valid"

    elif endingLine == -2:
        df.loc[indexFolder, 'generatedResultPegasus'] = "Empty Method"    

    else:
        result = '\n'.join(perturbed_pegasus_Java_Result.splitlines()[startingMethodLine:endingLine])
        resultsPegasusList.append(result)
        df.loc[indexFolder, 'generatedResultPegasus'] = result


#     ######################################################################################

    path_to_pivoting = generated_instances_path + str(indexFolder) + '/' + 'PerturbedPivoting.java'
    with open(path_to_pivoting) as f:
        perturbed_pivoting_Java = f.read()

    path_to_pivoting_result = generated_instances_path + str(indexFolder) + '/' + 'ResultPerturbedPivoting.java'
    with open(path_to_pivoting_result) as f:
        perturbed_pivoting_Java_Result = f.read()


    if(not sanitySaveCheck(perturbed_pivoting_Java_Result, 'ResultPerturbedPivoting')):
        print(indexFolder)

    endingLine = extractGeneratedMethod(perturbed_pivoting_Java_Result, startingMethodLine, endingMethodLine)
    #print('--> ',endingLine)
    if endingLine == -1:
        resultsPivotingList.append("Not Valid")
        df.loc[indexFolder, 'generatedResultPivoting'] = "Not Valid"

    elif endingLine == -2:
        df.loc[indexFolder, 'generatedResultPivoting'] = "Empty Method"

    else:
        result = '\n'.join(perturbed_pivoting_Java_Result.splitlines()[startingMethodLine:endingLine])
        resultsPivotingList.append(result)
        df.loc[indexFolder, 'generatedResultPivoting'] = result


        ######################################################################################




893it [00:04, 196.92it/s]


In [12]:
df.to_csv('NonFullContextResult.csv')